<h2>Implementacja modelu HCRNN do postaci Neurona</h2>

Artykuł naukowy:
Instniejace, podobne rozwiązania:

In [ ]:
import numpy as np
import math, os
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch import Tensor

In [ ]:
__all__ = [
    'HCRNN_Neuron'
]

In [ ]:
from torch.nn import Module

'''
Klasa rozszerzająca funkcje torch'a.
'''

class HCRNN_Neuron(nn.Module):

    '''
    definicja stałych określająca rozmiar wejścia i wyjścia dla liczb neuronów.
    '''

    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int

    def __init__(
        self,
        *,
        in_features: int,
        out_features: int,
        
        #definicja urządzenia, CPU, GPU lub TPU
        device = None,
        dtype= None,

    ) -> None:
        factory_kwargs = {"device": device, "dtype": dtype}
        super().__init__()

    #Tensor wchodzi i zawsze Tensor wychodzi
    def forward(self, input:Tensor) -> Tensor:
        pass

<h2>Normalization</h2>

In [ ]:
class CDFNorm(nn.Module):
    def __init__(
        self,
        *,
        unbiased: bool = True,
        eps: float = 1e-5,
        affine: bool = False,
    ):
        super().__init__()

        self.unbiased  = unbiased
        self.eps       = eps

        if affine:
            self.weight = nn.Parameter(torch.ones(1))
            self.bias   = nn.Parameter(torch.zeros(1))
        else:
            self.register_parameter("weight", None)
            self.register_parameter("bias",   None)

    def _empirical_cdf(self, x: torch.Tensor, dim: int) -> torch.Tensor:
        sorted_x, _ = torch.sort(x, dim=dim)
        idx = torch.searchsorted(sorted_x, x, right=False, dim=dim)
        N   = x.size(dim)
        return (idx.to(x.dtype) + 0.5) / N
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        stat_dim = 0
        
        u = self._empirical_cdf(x, stat_dim)

        if self.weight is not None:
            u = u * self.weight + self.bias
        return u

NameError: name 'nn' is not defined

<h2>Joint Distribution</h2>
Rozłożone w dwóch i trzech wymiarach.

In [ ]:
class JointDistribution3D(nn.Module):
    def __init__(self,
                 *,
                 x:Tensor,
                 y:Tensor,
                 z:Tensor,
                 a:Tensor
                 ):
        super().__init__()

        self.x = x,
        self.y = y,
        self.z = z,
        self.a = a
    
    #a = torch.rand((10,2))
    #b = torch.rand((10,5))

    def computeJointDistribution(self) -> Tensor:
        joint = torch.zeros()

        for i in range(2):
            for j in range(5):
                for z in range(5):
                    joint[:, i*5+j] = self.x[:, i]*self.y[:, j]*self.z[:, z]

        c = self.x.unsqueeze(1) @ self.y.unsqueeze(1) @ self.z.unsqueeze(1)
        
        torch.reshape(c, self.a.shape)

        # sprawdzić wyniki
        #print(torch.allclose(a, joint))

        return c

In [ ]:
class JointDistribution2D(nn.Module):
    def __init__(self,
                 *,
                 x:Tensor,
                 y:Tensor,
                 #z:Tensor
                 ):
        super().__init__()
    
    #a = torch.rand((10,2))
    #b = torch.rand((10,5))

    def computeJointDistribution(self) -> Tensor:
        joint = torch.rand((10,10))

        for i in range(2):
            for j in range(5):
                    joint[:, i*5+j] = self.x[:, i]*self.y[:, j]

        c = self.x.unsqueeze(1) @ self.y.unsqueeze(1)
        
        torch.reshape(c, self.a.shape)

        # sprawdzić wyniki
        #print(torch.allclose(c, joint))

        return c

<h2>Estymacja średnich</h2>

In [ ]:
class Estimation(nn.Module):
    def __init__(self,
                 *,
                 triplets,
                 feature_fn,
                 feature_dm
                 ):
        super().__init__()
        self.triplets = triplets,
        self.feature_fn = feature_fn,
        self.feature_dm - feature_dm

    def compute_tensor_mean(self) -> Tensor:
        """
        Parametry:
            triplets: array (x, y, z)
            feature_fn: funckaj mapująca
            feature_dim: wymiary D
        """
        a = np.zeros((self.feature_dim, self.feature_dim, self.feature_dim))
        
        for (x, y, z) in self.triplets:
            fx = self.feature_fn(x)
            fy = self.feature_fn(y)
            fz = self.feature_fn(z)
            
            outer = np.einsum(fx, fy, fz)
            
            a += outer

        a /= len(self.triplets)  # Normalizacja na trójkach
        return a

<h2>Estymacja warunkowa</h2>

In [ ]:
class ConditionalEstimation(nn.Module):
    def __init__(self, 
                 *,
                 x_candidates,
                 y,
                 z,
                 a,
                 feature_fn) -> None:
        super().__init__()
        self.x_candidates = x_candidates,
        self.y = y,
        self.z = z,
        self.a = a,
        self.feature_fn - feature_fn

    def conditional_score(self):

        D = self.a.shape[0]
        fy = self.feature_fn(self.y)
        fz = self.feature_fn(self.z)

        denominator = 0
        for j in range(D):
            for k in range(D):
                denominator += self.fa[0, j, k] * fy[j] * fz[k]

        scores = []
        for x in self.x_candidates:
            fx = self.feature_fn(x)
            
            score = 0
            for i in range(D):
                context_sum = 0
                for j in range(D):
                    for k in range(D):
                        context_sum += self.a[i, j, k] * fy[j] * fz[k]
                score += fx[i] * (context_sum / (denominator + 1e-8)) #uniknięcie dzielenia przez zero

            scores.append(score)

        return scores

<h2>Propagacja 1</h2>

In [ ]:
class PropagationEstimation(nn.Module):
    def __init__(self, 
                 *, 
                 y, 
                 z, 
                 a, 
                 feature_fn):
        super().__init__()
        self.y = y,
        self.z = z,
        self.a = a,
        self.feature_fn = feature_fn

    def propagate_expectation(self):

        fy = self.feature_fn(self.y)
        fz = self.feature_fn(self.z)
        D = fy.shape[0]

        numerator = 0.0
        denominator = 0.0
        for j in range(D):
            for k in range(D):
                numerator += self.a[1, j, k] * fy[j] * fz[k]
                denominator += self.a[0, j, k] * fy[j] * fz[k]

        propagated = 0.5 + (1 / (2 * np.sqrt(3))) * (numerator / (denominator + 1e-8))
        return propagated

<h2>Entropia</h2>

In [ ]:
class EntropyAndMutualInformation(nn.Module):

def approximate_entropy(activations):

    # Normalizacja prawdopodobieństw funkcji aktywacji
    probs = F.softmax(activations, dim=1)
    entropy = -torch.sum(probs ** 2, dim=1).mean()
    return entropy

def approximate_mutual_information(act_X, act_Y):

    # Normalizacja funkcji aktywacji
    probs_X = F.softmax(act_X, dim=1)
    probs_Y = F.softmax(act_Y, dim=1)
    
    joint_probs = torch.bmm(probs_X.unsqueeze(2), probs_Y.unsqueeze(1))
    
    mi = torch.sum(joint_probs ** 2, dim=(1,2)).mean()
    return mi

<h2>Dynamicznie modyfikowany model za pomocą EMA</h2>

In [ ]:
class DynamicEMA(nn.Module):
    def __init__(self, x, y, z, ema_lambda) -> None:
        self.x = x,
        self.y = y,
        self.z = z,
        self.ema_lambda = ema_lambda

    def EMAUpdateMethod(self):
        def f_i(x): return x
        def f_j(y): return y
        def f_k(z): return z

        update_tensor = torch.einsum('i,j,k->ijk', f_i(self.x), f_j(self.y), f_k(self.z))

        # EMA updating values
        a = (1 - self.ema_lambda) * a + self.ema_lambda * update_tensor
        
        return a

<h2>Optymizacja bazy</h2>

In [ ]:
class BaseOptimization(nn.Module):
    def __init__(self,
                 *,
                 a, #tensor do optymalizacji
                 ) -> None:
        self. a = a

    def optimization_early(self) -> Tensor:
        M = self.a.reshape(len(self.a[0]), -1)

        # Obliczenie SVD
        U, S, Vh = torch.linalg.svd(M, full_matrices=False)

        # Transformacja Bazy, tu przykładowa funkcja, do wymiany
        def f_x(x):
            return torch.sin(x * torch.linspace(0, 1, len(self.a[2])))

        # nowa baza g_i(x) = sum_j v_ij * f_j(x)
        def g_i(x, U):
            f = f_x(x)
            return torch.matmul(U.T, f)

        # Step 4: Transformacja Tensora
        new_a = torch.einsum('li,ljk->ijk', U.T, self.a)

        return new_a